In [46]:
from PIL import Image
import numpy as np
from scipy.fft import dct
from huffman import *
import math
import pickle

In [3]:
image = Image.open('photo1.png')
ycbcr = image.convert('YCbCr')

## Chroma Subsampling 4:2:2
npmat = np.array(ycbcr, dtype=np.uint8)
print(npmat)

chroma = npmat.copy()
chroma[:, 1::2] = chroma[:, ::2] # 4:2:2

[[[147 126 149]
  [146 126 149]
  [148 126 149]
  ...
  [ 99 134 145]
  [101 134 145]
  [103 134 143]]

 [[139 126 149]
  [141 126 149]
  [150 126 149]
  ...
  [ 98 134 145]
  [100 134 145]
  [102 134 143]]

 [[149 126 150]
  [143 126 150]
  [146 126 149]
  ...
  [ 99 134 145]
  [101 134 145]
  [102 134 143]]

 ...

 [[  4 130 128]
  [  0 130 127]
  [  6 130 128]
  ...
  [ 46 101 131]
  [124  88 133]
  [127  85 133]]

 [[  5 130 128]
  [  1 130 128]
  [  7 130 128]
  ...
  [  7 124 127]
  [ 74  86 132]
  [131  83 133]]

 [[  6 130 128]
  [  4 130 128]
  [  9 130 128]
  ...
  [109  91 132]
  [139  85 133]
  [ 95  81 133]]]


In [4]:
quan_lum = np.array([[16,11,10,16,24,40,51,61],
                    [12,12,14,19,26,58,60,55],
                    [14,13,16,24,40,57,69,56],
                    [14,17,22,29,51,87,80,62],
                    [18,22,37,56,68,109,103,77],
                    [24,35,55,64,81,104,113,92],
                    [49,64,78,87,103,121,120,101],
                    [72,92,95,98,112,100,103,99]])
# Chrominance
quan_chr = np.array([[17,18,24,47,99,99,99,99],
                    [18,21,26,66,99,99,99,99],
                    [24,26,56,99,99,99,99,99],
                    [47,66,99,99,99,99,99,99],
                    [99,99,99,99,99,99,99,99],
                    [99,99,99,99,99,99,99,99],
                    [99,99,99,99,99,99,99,99],
                    [99,99,99,99,99,99,99,99]])
def quantization(x, type):

    assert type in ('lum', 'chr')
        
    if type == 'lum':
        return np.round(x/quan_lum)
    else:
        return np.round(x/quan_chr)


In [5]:
def run_length_coding(x):
    rlc = []
    last_iteratation = 0
    for i, j in enumerate(x):
        ##print('x', j)
        if j != 0:
            rlc.append([i-last_iteratation, j])
            last_iteratation = i+1
            ##print(rlc)

    return rlc


In [6]:
def dpcm(x):
    dpcm = []
    dpcm.append(x[0])
    for i in range(1, len(x), 1):
        dpcm.append(x[i] - x[i-1])

    return dpcm

In [24]:
def bit_numerator(x):
    if x ==0:
        return 1
    if x < 0:
        x = x * -1
        return 1 + math.floor(math.log(x, 2))
    if x > 0:
        ##print((math.log([x,2])))
        return math.ceil(math.log(x,2))


In [8]:
def go_up(point):
    return (point[0] - 1, point[1])
def go_down(point):
    return (point[0] + 1, point[1])
def go_right(point):
    return (point[0], point[1] + 1)
def go_left(point):
    return (point[0], point[1] - 1)

def check_border(point, row, col):
    return 0 <= point[0] < row and 0 <= point[1] < col

def zigzag(row, col):
    point = (0, 0)
    total_block = row * col
    ##print(total_block)
    f = True
    ##print('hi')
    for _ in range(row * col):
        ##print(point)
        yield point
        if f:
            if check_border(go_up(go_right( point)), row, col):
                point = go_up(go_right( point))
            else:
                f = False
                if check_border(go_right( point), row, col):
                    point = go_right( point)
                else:
                    point = go_down(point)
        else:
            if check_border(go_down(go_left(point)), row, col):
                point = go_down(go_left(point))
            else:
                f = True
                if check_border(go_down( point), row, col):
                    point = go_down( point)
                else:
                    point = go_right( point)


In [9]:

block_y = 8
block_x = 8


# Vertical padding
if chroma.shape[0] % block_y != 0:
    print('---------Vertical--------')
    vpad = chroma.shape[0] % block_y
    vpad = block_y - vpad
    top= vpad // 2 
    bottom = vpad - top
    image = np.concatenate((np.repeat(chroma[:1], top, 0), chroma, 
                            np.repeat(chroma[-1:], bottom, 0)), axis=0)
        
# Horizontal padding
if chroma.shape[1] % block_x != 0:
    hpad = chroma.shape[1] % block_x
    left = hpad // 2 
    right = hpad - left
    image = np.concatenate((np.repeat(chroma[:,:1], left, 1), chroma, 
                            np.repeat(chroma[:,-1:], right, 1)), axis=1)

total_blocks = (image.shape[0]//8 ) * (image.shape[1]//8)
print(image.shape[0], image.shape[1])
dc = np.empty((total_blocks, 3), dtype=np.int32)
ac = np.empty((total_blocks, 63, 3), dtype=np.int32)

blocks = []
counter = 0
for i in range(0, image.shape[0], 8):
    for j in range(0, image.shape[1], 8):
        counter += 1
        for k in range(3):
            blocks.append({'block':image[i:i+block_x, j:j+block_y, k],
                            'k': k,
                            'counter': counter})
print(counter)           
for block in blocks:
    dc_m = dct(dct(block['block'].T, norm='ortho').T, norm='ortho')
    qua_m = quantization(dc_m, 'lum' if block['k'] == 0 else 'chr')
    ##print(qua_m.shape)
    z = np.array([qua_m[point] for point in zigzag(*qua_m.shape)])
    ##print(z)
    dc[block['counter'] - 1, block['k']] = z[0]
    ac[block['counter'] - 1, :, block['k']] = z[1:]



---------Vertical--------
856 1280
17120


In [40]:
rlc_lum = []
rlc_chr = []

for b in range(counter):
    for x in run_length_coding(ac[b,:,0]):
        if len(x)!=0:
            rlc_lum.append(x)

for b in range(counter):
        for i in range(1,2,1):
            for x in run_length_coding(ac[b,:,i]):
                if len(x)!=0:
                    rlc_chr.append(x)



In [55]:
x = dpcm(dc[:, 0])
dc_lum = []
for i in x:
    dc_lum.append(int32(bit_numerator(int(i))))

x = dpcm(dc[:, 1:].flat)
dc_chr = []
for i in x:
    dc_chr.append(int32(bit_numerator(int(i))))

dc_huffman_lum = encode(dc_lum)
dc_huffman_chr = encode(dc_chr)

7_6_5_4_3_0_2_1
<class 'huffman.Node'>
7_0_6_1_2_5_3_4
<class 'huffman.Node'>


In [51]:
ac_lum = [int32(rlc_lum[i][0]) for i in range(len(rlc_lum)-1)]
ac_chr = [int32(rlc_chr[i][0]) for i in range(len(rlc_chr)-1)]

ac_huffman_lum = encode(ac_lum)
ac_huffman_chr = encode(ac_chr)

20_30_28_27_29_26_25_24_21_23_22_19_18_17_16_15_14_13_12_11_10_9_8_7_6_5_4_3_2_1_0
<class 'huffman.Node'>
10_8_38_13_9_7_6_5_4_3_2_1_0
<class 'huffman.Node'>


In [52]:
def write_to_file(name, x):
    file = open('{}.obj'.format(name), 'wb') 
    pickle.dump(x, file)
    file.close()

In [58]:
file = 'output'

write_to_file(file, dc_huffman_lum)
write_to_file(file, dc_huffman_chr)
write_to_file(file, ac_huffman_lum)
write_to_file(file, ac_huffman_chr)
write_to_file(file, rlc_chr)
write_to_file(file, rlc_lum)